In [ ]:
#Projet Mogpl M1 Informatique
#Dice Battle

#Yuhan WANG & Yufei LIU

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import math

#Q1
def Q(d,k):
    if k<=0 | ((k>1) & (k<2*d)) | k>6*d :
        return 0
    elif d==1:
        if k>1 & k<7:
            return 1/5
        else:
            return 0
    else:
        res = 0
        for j in range(2,7):
            res += Q(d-1,k-j)
        return res/5

print(Q(2,8))
        

def calcul_tableau_P(d,k):
    values_possibles = [ i for i in range(1,k+1) ]
    res = np.zeros((d , k))
    #d2 = 2*d
    for i in range(d):
        for j in range(k):
            ii = i+1
            jj = j+1
            if jj == 1:
                res[i][j] = 1 - (5/6)**ii
            elif jj < 2*ii:
                res[i][j] = 0
            else:
                res[i][j] = (5/6)**ii * Q(ii,jj)
    return res
    
print(calcul_tableau_P(5,20))

0.2
[[1.66666667e-01 1.66666667e-01 1.66666667e-01 1.66666667e-01
  1.66666667e-01 1.66666667e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.05555556e-01 0.00000000e+00 0.00000000e+00 2.77777778e-02
  5.55555556e-02 8.33333333e-02 1.11111111e-01 1.38888889e-01
  1.11111111e-01 8.33333333e-02 5.55555556e-02 2.77777778e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.21296296e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.62962963e-03 1.38888889e-02 2.77777778e-02
  4.62962963e-02 6.94444444e-02 8.33333333e-02 8.79629630e-02
  8.33333333e-02 6.94444444e-02 4.62962963e-02 2.77777778e-02
  1.38888889e-02 4.62962963e-03 0.00000000e+00 0.00000000e+00]
 [5.17746914e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 